In [2]:
import psycopg2
from psycopg2 import Error
import pandas as pd
import random
import datetime
from datetime import timedelta
import string

# количество записей
male = 735 * ['М']
female = 938 * ['Ж']
gender = male + female
N = 200
#создаем список имен
namedf = pd.read_csv('name.csv', usecols = ['name'])
namedf.insert(loc = 1, column = 'gender', value = gender)

#создаем список фамилий
surnamedf = pd.read_csv('surname.csv', usecols = ['surname'])
surname = surnamedf['surname']
#создаем паспорта
passport = [(str(random.randint(1000, 9999)) + " " + str(random.randint(100000, 999999))) for i in range(N)]
#создаем список номеров телефонов
phone = ['89' + str(random.randint(0,999999999)) for i in range(N)]
#создаем список номеров паролей
password = [str(random.randint(10000000, 10000000000000)) for i in range(N)]
#создаем список номеров льгот
fees = [random.choice(['TRUE', 'FALSE']) for i in range(N)]
#создаем список дат рождения
start_date = datetime.date(1900, 1, 1)
end_date = datetime.date(2023, 1, 1)
def random_date(start_date, end_date):
    random_date = start_date + datetime.timedelta(days=random.randint(0, (end_date - start_date).days))
    date = f"'{random_date.strftime('%Y-%m-%d')}'"
    return date

born_date = [random_date(start_date, end_date) for i in range(N)]
    
#создаем список email

email = []
def random_string(length):
    letters = string.ascii_lowercase
    rand_string = ''.join(random.choice(letters) for i in range(length))
    return rand_string
em = ['yandex.ru', 'gmail.ru', 'mail.ru']
for i in range(N):
    email.append(random_string(8) + '@' + random.choice(em))
    
#создаем списко логинов
login = [random_string(8) for i in range(N)]
try:
    # Подключиться к существующей базе данных
    connection = psycopg2.connect(user="azonchic",
                                  # пароль, который указали при установке PostgreSQL
                                  password="rgdbaza",
                                  host="127.0.0.1",
                                  port="5432",
                                  database="rgd")

    # Создайте курсор для выполнения операций с базой данных
    cursor = connection.cursor()
    for i in range(N):
        num = str(i + 1)
        # выбираем случайное имя и фамилию
        k = random.randint(0, len(namedf) - 1)
        name = namedf.iloc[k]
        n = random.randint(0, len(surname) - 1)
        gender = namedf.iat[i,1]
        insert_query = """INSERT INTO Passenger (id, name, surname, 
                passport, phone_number, 
                password1, fees, born_date, 
                email, gender, login) VALUES (%s, %s, %s, %s,
                  %s, %s,
                  %s, %s, %s,
                  %s, %s)"""
        cursor.execute(insert_query, (i + 1, name[0], surname[n] + '(a)', passport[i],
                        phone[i], password[i], fees[i], born_date[i], email[i],
                                     gender, login[i]))
        connection.commit()
    # Выполнение команды: это создает новую таблицу
except (Exception, Error) as error:
    print("Ошибка при работе с PostgreSQL", error)
finally:
    if connection:
        cursor.close()
        connection.close()
        print("Соединение с PostgreSQL закрыто")

Соединение с PostgreSQL закрыто
